In [89]:
import requests
from bs4 import BeautifulSoup
import string
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from pymorphy2 import MorphAnalyzer
from nltk.util import ngrams
from collections import Counter
import csv
from sklearn.metrics import accuracy_score

К сожалению, у меня не получилось спарсить отзывы. 
Отзовик не дает доступ к исходному коду, на IRecommend есть ограничение в 10 отзывов за раз. Даже прокси не помогает. Ниже приведен написанный парсер для IRecommend. 

In [213]:
#место для ссылок и метаинфы
#ссылки на плохие отзывы, хорошие отзывы и какое-то количество рандомных
bad_link = 'https://irecommend.ru/content/balzam-opolaskivatel-bioaktivnyi-balzam-opolaskivatel-khors-fors-loshadinaya-sila?ft[r]=0'
good_link = 'https://irecommend.ru/content/balzam-opolaskivatel-bioaktivnyi-balzam-opolaskivatel-khors-fors-loshadinaya-sila?ft[r]=1&new=1'
random_link = 'https://irecommend.ru/content/balzam-opolaskivatel-bioaktivnyi-balzam-opolaskivatel-khors-fors-loshadinaya-sila'

#множества слов, встречающихся только в плохих и только в хороших отзывах
bad_words = []
good_words = []

#тексты отзывов
bad_reviews = []
good_reviews = []
random_reviews = []


In [43]:
#функция для препроцессинга
#чистим от знаков препинания, стоп-слов, лемматизируем
def preprocess(review):
    review = review.replace('\n', '').lower()
    
    for elem in punct:
        if elem in review:
            review = review.replace(elem, ' ')
    re.sub(r' {2,}', ' ', review)
    
    word_tokens = word_tokenize(review)
    review = ''
    for word in word_tokens:
        if word not in stop_words:
            word = morph.parse(word)[0].normal_form
            review += word + ' '
        
    return review

In [229]:
def get_data(link, reviews_list, i):
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    print(response)
    
    #находим все ссылки на конкретные отзывы
    review_data = soup.find_all('div', {'class': 'reviewTitle'})
    
    source_links = []
    for elem in review_data:
        elem = elem.find('a')
        try:
            source_links.append(elem.get('href'))
        except AttributeError as e:
            pass
        
    if len(source_links) > i:
        sourse_links = source_links[:i]
    
    #собираем текст отзыва, делаем препроцессинг
    for adress in source_links:
        site = 'https://irecommend.ru' + adress
        response = requests.get(site)
        soup = BeautifulSoup(response.text)
        review = soup.find('div', {'class':'views-field-teaser reviewText'})
        review = preprocess(review)
        reviews_list.append(review)

In [230]:
get_data(bad_link, bad_reviews, 100)
get_data(good_link, good_reviews, len(bad_reviews))
get_data(random_link, random_reviews, 10)

<Response [521]>
<Response [521]>
<Response [521]>


Я нашла датасет с отзывами на женскую одежду и аксессуары, буду брать данные оттуда. Они размечены как положительные/отрицательные. Положительные - 4-5 звезд, отрицательные - 1-2 звезды. Еще кажется, что это данные с алиэкспресса. Это, наверное, может как-то повлиять.

In [50]:
negative_raw = []
positive_raw = []

with open('women-clothing-accessories.csv', encoding = 'utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        row = row[0].split(';')
        try:
            if row[1] == 'negative':
                negative_raw.append(row[0])
            if row[1] == 'positive':
                positive_raw.append(row[0])
        except IndexError as e:
            pass

In [51]:
%time

negative = []
positive = []

morph = MorphAnalyzer()
punct = string.punctuation
stop_words = set(stopwords.words('russian'))

for review in negative_raw:
    negative.append(preprocess(review))
    
for review in positive_raw:
    positive.append(preprocess(review))

Wall time: 0 ns


In [61]:
negative = ''.join(negative)
negative = word_tokenize(negative)
n = set(negative)
               
positive = ''.join(positive)
positive = word_tokenize(positive)
p = set(positive)

only_positive = p.difference(n) # слова, которые есть только в позитивных отзывах
only_negative = n.difference(p) # слова, которые есть только в негативных отзывах

negative = dict(Counter(negative))
negative_clear = [] 
for elem in negative:
    if elem in only_negative and negative[elem] > 2:
        negative_clear.append(elem)
        
positive = dict(Counter(positive))
positive_clear = []
for elem in positive:
    if elem in only_positive and positive[elem] > 2:
        positive_clear.append(elem)

Итого, у нас есть списки не особо редких(больше двух вхождений) слов, котрые встречаются только в положительных и только в отрицательных отзывах на женскую одежду. Я скопировала с IRecommend 10 отзывов на женские чулки, блузки и штаны - разной тональности и на них буду проверять.

In [80]:
gold = []
some_reviews = []

with open('some_reviews.txt', encoding = 'utf-8') as f:
    text = f.read()
    
text = text.split('\n--\n')
for elem in text:
    elem = elem.split(':\n')
    gold.append(elem[0])
    some_reviews.append(preprocess(elem[1]))

In [85]:
results = []

for review in some_reviews:
    good_count = 0
    bad_count = 0
    review = word_tokenize(review)
    for word in review:
        if word in positive_clear:
            good_count += 1
        if word in negative_clear:
            bad_count += 1
    if bad_count > good_count:
        results.append('negative')
    elif bad_count < good_count:
        results.append('positive')
    else:
        results.append('no result')
    

In [87]:
accuracy_score(gold, results)

0.5